# `database 의 index`

sql 튜닝을 공부하던 중 가장 기초적인 내용이 머리 속에서 흩어져 내가 다시 공부하려고 쓰는 파일이다.

index 는 쿼리의 성능을 올리기 위해 사용한다. (빠른 쿼리, 정렬, 조건 최적화 등등등..)

전에 spark 에서 hdfs 로 파일을 옮기기 위해 db 를 하나 만든 게 있는데 그거로 예시를 들어보겠다.

<br>| Field         | Type        | Null | Key | Default | Extra          |
<br>| idx           | int         | NO   | PRI | NULL    | auto_increment |
<br>| summoner_id   | varchar(60) | YES  |     | NULL    |                |
<br>| summoner_name | varchar(20) | YES  |     | NULL    |                |
<br>| QType         | varchar(20) | YES  |     | NULL    |                |

위와 같은 desc table 정보가 있고 데이터는 600개 정도 된다. 

만약 인덱스를 활용하지 않고 1624 라는 닉네임을 가진 유저를 찾는다 하면 어떻게 될까?

답은 모든 데이터를 한 번씩 확인하는 full table scan 을 하게된다.

`유저 한 명을 찾기위해 데이터베이스에 있는 모든 유저를 한 번씩 탐색한다`는 뜻이고, 시간 복잡도는 O(N) 이 되겠다.

<img src = 'https://drive.google.com/uc?id=1QzW0ow05qhxeCK19wuIcbcRAAhw0z7PM' width = 600, height = 100>

1 row in set (0.0021 sec) 이 걸렸다.

데이터의 양이 방대해지면 full table scan 시 많은 시간이 필요할 것이다.

반응이 중요한 곳에서는 쓸 수 없는 쿼리가 된다. 

다음은 인덱스를 적용해서 `create index s_name on user_info(summoner_name);``

똑같은 쿼리문으로 얼마의 시간이 걸리는지 보면...

<img src = 'https://drive.google.com/uc?id=1ZPKDHpZnxOWKT169175QcD97PGqx7zKS' width = 600, height = 100>

1 row in set (0.0006 sec) 이 걸렸다.

총 데이터의 양이 980 개 임에도 3.5 배의 효율을 이끌어 냈다. 

mysql 에서 index 는 기본적으로 B-tree(balanced tree) 구조이고, 정렬이 된 상태이다. 

<img src = 'https://drive.google.com/uc?id=1Wn9Mm4RWAc-kZ0FwJWJibx_khFBOn_UP' width = 550, height = 400>

얘를 들어 5의 숫자를 찾고 싶으면, 10 보다 작은 숫자가 있는 sub-tree 로 가서 숫자를 찾는다. 

위에서는 name 이라는 컬럼에 index 를 적용해 b-tree 구조로 만들어서 숫자와 글이 합쳐져 있지만, 크게 다르지 않다.

b - tree 이름에서 알 수 있는 균형잡힌 tree 이다.

리프 노드까지의 깊이가 비슷하게 유지되고, 높이가 늘어나는 것을 방지한다.

한 번의 탐색으로 여러 데이터에 접근하고 탐색 과정에서 거쳐야 하는 노드의 개수가 밑의 제곱 수로 늘어날 때마다 1 씩 증가한다.

따라서 시간 복잡도는 O(log n) 이다. `데이터의 양이 아무리 많아져도 tree 의 높이가 느리게 증가하고, y 값은 특정 값에 수렴하게 된다.`

다른 database 에서 학생들의 키와 몸무게에 대한 정보를 담고있고 그 정보들을 쿼리한다고 가정하자.

키에 index 를 지정한 후 where 에 x between y, select avg(weight) 를 쿼리하면 index 를 사용했음에도 성능이 좋아지지 않는다.

그 이유는 키에는 index 가 있어도 몸무게는 없기 때문에 

x < 키 < y 에 해당하는 학생이 만약 100 만명이라면 100 만명 모두의 몸무게를 탐색한다. (index 를 활용하지 않았음.)

그래서 위와 같은 경우는 키와 몸무게에 인덱스를 걸어서 (키, 몸무게) 이런 식으로 탐색될 수 있게 해준다. (composite index)

`create index stu_hei,wei on student(height, weight);` 이 코드와 같이 써서 키 인덱스에 몸무게가 있도록 해준다.

그러면 쿼리의 성능이 높아질 것이다. 

그리고 composite index 시에는 뭐가 앞에 올 것인지에 대한 순서도 매우 중요한다.

(키, 몸무게) 순으로 인덱스를 걸었다면, 키 컬럼의 값으로 먼저 필터링하고 몸무게 컬럼의 값으로 필터링한다.

정확한 내용을 까먹어서... 나중에 나를 위해 정리한다.